# RoBERTa:
**Model Size:**


RoBERTa is typically larger than DistilBERT, especially the "Large" variant. It has more parameters, making it potentially more powerful but also computationally more expensive.


**Training Techniques**:


RoBERTa uses advanced training techniques, such as removing the Next Sentence Prediction (NSP) objective and training with larger mini-batches. These modifications often lead to improved performance.


**Training Data**:


RoBERTa is trained on a large corpus of text data, and it benefits from extensive pre-training. The training data and techniques contribute to its robustness and effectiveness across various NLP tasks.


**Performance:**


RoBERTa is known for achieving state-of-the-art results on a wide range of natural language processing benchmarks. Its performance is generally top-tier, especially for tasks like text classification, sentiment analysis, and named entity recognition.

# Load Libraries
**Changes:**


-Using RoBerta instead DistillBert


-fix the test data predicting

In [1]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade

# This sample uses Keras Core, the multi-backend version of Keras.
# The selected backend is TensorFlow (other supported backends are 'jax' and 'torch')
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras_core as keras
import keras_nlp
import seaborn as sns
import matplotlib.pyplot as plt

train_df = pd.read_csv('/kaggle/input/artificial-text-detection-homework/dev.csv')

X = train_df['Text']
y = train_df['Class'].apply(lambda x: 1 if x == 'M' else 0)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using TensorFlow backend


# Load The DistilBert for classification

In [3]:
# Load a DistilBERT model.
preset= "roberta_base_en"

# Use a shorter sequence length.
preprocessor = keras_nlp.models.RobertaPreprocessor.from_preset(preset,
                                                                   sequence_length=160,
                                                                   name="preprocessor_4_tweets"
                                                                  )

# Pretrained classifier.
classifier = keras_nlp.models.RobertaClassifier.from_preset(preset,
                                                               preprocessor = preprocessor, 
                                                               num_classes=2)

classifier.summary()

898823/898823 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step       
456318/456318 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step       
496436344/496436344 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Preprocessor: "preprocessor_4_tweets"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ roberta_tokenizer (RobertaTokenizer)               │                                              50,265 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "roberta_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃     Param # ┃ Connected to                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │           0 │ -                              │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │           0 │ -                              │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ roberta_backbone              │ (None, None, 768)         │ 124,052,736 │ padding_mask[0][0],            │
│ (RobertaBackbone)             │                           │             │ token_ids[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ get_item (GetItem)            │ (None, 768)               │           0 │ roberta_backbone[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ pooled_dropout (Dropout)      │ (None, 768)               │           0 │ get_item[0][0]                 │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ pooled_dense (Dense)          │ (None, 768)               │     590,592 │ pooled_dropout[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ classifier_dropout (Dropout)  │ (None, 768)               │           0 │ pooled_dense[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ logits (Dense)                │ (None, 2)                 │       1,538 │ classifier_dropout[0][0]       │
└───────────────────────────────┴───────────────────────────┴─────────────┴────────────────────────────────┘

 Total params: 124,644,866 (475.48 MB)

 Trainable params: 124,644,866 (475.48 MB)

 Non-trainable params: 0 (0.00 B)

# Prepare Data

In [4]:
def remove_br_tags(text):
    return text.replace('<br />', '')

X = X.apply(remove_br_tags)

In [5]:
from sklearn.model_selection import train_test_split
# Assuming train_X and train_y are your features and labels
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Compile The Model

In [6]:
# Compile
classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), #'binary_crossentropy',
    optimizer=keras.optimizers.Adam(1e-5),
    metrics= ["accuracy"]  
)

# Train (Finetune) DistilBertForClassification

In [7]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience = 60, 
    min_delta = 1e-4, 
    restore_best_weights = True
)

In [8]:
EPOCHS = 100
BATCH_SIZE = 32

# Fit
history = classifier.fit(x=X_train,
                         y=y_train,
                         batch_size=BATCH_SIZE,
                         epochs=EPOCHS, 
                         validation_data=(X_val, y_val), 
                         callbacks= [early_stopping]
                        )

Epoch 1/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 121s 753ms/step - accuracy: 0.7094 - loss: 0.5835 - val_accuracy: 0.9875 - val_loss: 0.0453
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 31s 579ms/step - accuracy: 0.9806 - loss: 0.0615 - val_accuracy: 1.0000 - val_loss: 0.0053
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 30s 578ms/step - accuracy: 0.9890 - loss: 0.0227 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 30s 574ms/step - accuracy: 1.0000 - loss: 0.0036 - val_accuracy: 1.0000 - val_loss: 9.7034e-04
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 31s 594ms/step - accuracy: 1.0000 - loss: 0.0034 - val_accuracy: 1.0000 - val_loss: 0.0022
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 30s 574ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 1.0000 - val_loss: 8.3761e-04
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 30s 576ms/step - accuracy: 1.0000 - loss: 8.6383e-04 - val_accuracy: 1.0000 - val_loss: 4.4874e-04
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 30s 563ms/step - accuracy: 1.0000 

# Predict on test

In [9]:
df_test = pd.read_csv('/kaggle/input/artificial-text-detection-homework/test.csv')
df_test['Text'] = df_test['Text'].apply(remove_br_tags)

df_test['Class'] = [('M' if np.argmax(i) == 1 else 'H') for i in classifier.predict(df_test['Text'].to_list())]

625/625 ━━━━━━━━━━━━━━━━━━━━ 87s 130ms/step


In [10]:
submission_df = pd.DataFrame({'ID': df_test['ID'], 'Class': df_test['Class']})
submission_df.to_csv('submission.csv', index = False)

# Upvote and Comment if you like this notebook😉